* 최소한 weights를 저장하기 전에 모든 코드실행을 한 후에 제출하고 싶었으나  Training을 할 때 문제가 발생했습니다.
* 코칭시간에 가서 해결을 해야 할 것 같습니다.
* tensorboard에 tensor를 추가할 때 문제가 생기는 것 같습니다.
* 빈칸을 넣는데에 시간을 굉장히 많이 썼습니다. 수업시간에 한 코드와 약간은 상이해서 난관이 많았습니다.
* 일단 training을 하는 과정에서 saver를 이용해 min validation model을 저장하고 불러온 후 min loss와 같은 값을 보이는 지 확인해야 합니다.

## CIFAR-10, 100 학습시키기

## Objective

1.[CIFAR -10 Data](https://www.cs.toronto.edu/~kriz/cifar.html) 을 Convolution Neural Network 을 이용해 학습해봅니다.
----
![Imgur](https://i.imgur.com/yy09iLz.png)


- loss 가 가장 작은 model 을 저장합니다.
- 목표 accuracy 는 75% 입니다. 
​

------


In [0]:
import numpy as np 
import tensorflow as tf 
import matplotlib.pyplot as plt
import os

# Load Cifar-10 dataset 
 - cifar 10 dataset 을 다운로드 합니다. 
 - normalize 을 수행합니다.

In [0]:
# load cifar10 dataset 
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# reshape (None, 1) -> (None)
y_train, y_test = [y.reshape([-1]) for y in (y_train, y_test)]

# normalization 
x_train, x_test = [x /255. for x in (x_train, x_test)]

# N class
n_classes = 10
print('image shape : {}, label shape : {} '.format(x_train.shape, y_train.shape))
print('image shape : {}, label shape : {} '.format(x_test.shape, y_test.shape))
print('train minimun : {}, train_maximum : {} '.format(x_train.min(), x_train.max()))
print('tests minimun : {}, test_maximum : {} '.format(x_test.min(), x_test.max()))

Using TensorFlow backend.


170500096/170498071 [==============================] - 3s 0us/step
image shape : (50000, 32, 32, 3), label shape : (50000,) 
image shape : (10000, 32, 32, 3), label shape : (10000,) 
train minimun : 0.0, train_maximum : 1.0 
tests minimun : 0.0, test_maximum : 1.0 


# DataProvider 

In [0]:
import numpy as np
import random

class DataProvider(object):
    def __init__(self, images, labels):
        self.n_sample = len(labels)
        self.queue = list(range(self.n_sample))
        random.shuffle(self.queue)

        self.images = images
        self.labels = labels
        self.epoch_count = 0

    def next_batch(self, batch_size):
        if len(self.queue) < batch_size:
            self.queue = list(range(self.n_sample))
            self.epoch_count += 1
        target_indices = self.queue[:batch_size]
        del self.queue[:batch_size]
        return self.images[target_indices], self.labels[target_indices]

# Configuration

설계한 모델을 표로 작성합니다. 

- 목표 Receptive Field : ? <br>
- Convolution Phase 후  출력 크기  :  ? <br>


| 층  | 종류|필터 갯수  | 필터 크기 | 스트라이드 | 패딩   | Dropout | output size |
|--- |--- |----|----|----|----|----| ---| 
| ? |?| ?|? |?  | ? |?| ?|


- 모델 설계가 끝나면 간단한 그림을 작성해 아래에 붙여주세요.


예시1) 


- 목표 Receptive Field : 28 <br>
- Convolution Phase 후  출력 크기  :  4 <br>
- Regularization  : L2 
- Batch size : 120
- Learning rate : 0.0001 
- Data normalization : min max normalization 
- Standardization : None 


| 층  | 종류|필터 갯수  | 필터 크기 | 스트라이드 | 패딩   | Dropout | output size |
|--- |--- |----|----|----|----|----| ---| 
| c1 |conv| 64| 3x3| 1  | SAME | None| 32x32 |
| s2 |max-pooling| None| 3x3| 2  | SAME | None|16x16 | 
| c3 |conv| 128| 3x3| 2  | SAME |NOne |16x16 | 
| s4 |max-pooling| None| 3x3| 2  | SAME | None|8 x8 | 
| c5 |conv| 128| 3x3| 2  | SAME | None |8 x8 | 
| s6 |conv| 256| 3x3| 2  | SAME | None |4 x 4 | 
| c7 |conv| 256| 1x1| 2  | SAME | None |4 x 4 | 
| f8 ||| | FC 256  | |  || 
| f8 ||| | Dropout 0.7 | |  || 
| f9 ||| | FC 256  | |  || 
| f9 ||| | Dropout 0.6 | |  || 
| f10||| | FC 10   | |  || 




![Imgur](https://i.imgur.com/yqrIm5u.png)

# Convolution layer
- convolution layer helper function 을 정의합니다.
- 위 설계한 convolution layer 을 구현합니다 

In [0]:
# convolution helper function
def conv(input_xs ,units, k, s, padding, activation, name):
    with tf.variable_scope(name, reuse=tf.AUTO_REUSE):
        ch = int(input_xs.get_shape()[3])
        ws = tf.get_variable(name='ws', shape=[k,k,ch,units], dtype=tf.float32, initializer=tf.initializers.he_normal())
        b = tf.get_variable(name='bias', initializer=tf.zeros(shape=[units], dtype=tf.float32))
        layer = tf.nn.conv2d(input_xs, ws, [1,s,s,1], padding=padding, name='conv') + b
    return layer

In [0]:
# define input placeholder 
xs = tf.placeholder(shape=[None, 32, 32, 3], dtype=tf.float32, name='xs')
ys = tf.placeholder(shape=[None], dtype=tf.int32, name='ys')
lr = tf.placeholder(shape=[], dtype=tf.float32, name='lr')
phase_train = tf.placeholder(shape=[], dtype=tf.bool, name='phase_train')

# Model implementation 
# convolution Neural Network 
# 자신이 설계한 모형을 구현해주세요. 
layer1 = conv(xs, units=8, k=3, s=1, padding='SAME', activation=tf.nn.relu, name='layer_1')
layer2 = conv(layer1, units=8, k=3, s=1, padding='SAME', activation=tf.nn.relu, name='layer_2')
max_pool1 = tf.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), name='maxpool_1')(layer2)

layer3 = conv(max_pool1, units=16, k=3, s=1, padding='SAME', activation=tf.nn.relu, name='layer_3')
layer4 = conv(layer3, units=16, k=3, s=1, padding='SAME', activation=tf.nn.relu, name='layer_4')
max_pool2 = tf.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), name='maxpool_2')(layer4)

layer5 = conv(max_pool2, units=32, k=3, s=1, padding='SAME', activation=tf.nn.relu, name='layer_5')
layer6 = conv(layer5, units=32, k=3, s=1, padding='SAME', activation=tf.nn.relu, name='layer_6')
layer7 = conv(layer6, units=32, k=3, s=1, padding='SAME', activation=tf.nn.relu, name='layer_7')
max_pool3 = tf.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), name='maxpool_3')(layer7)

layer8 = conv(max_pool3, units=64, k=3, s=1, padding='SAME', activation=tf.nn.relu, name='layer_8')
layer9 = conv(layer8, units=64, k=3, s=1, padding='SAME', activation=tf.nn.relu, name='layer_9')
layer10 = conv(layer9, units=64, k=3, s=1, padding='SAME', activation=tf.nn.relu, name='layer_10')
max_pool4 = tf.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), name='maxpool_4')(layer10)

layer11 = conv(max_pool4, units=128, k=3, s=1, padding='SAME', activation=tf.nn.relu, name='layer_11')
layer12 = conv(layer11, units=128, k=3, s=1, padding='SAME', activation=tf.nn.relu, name='layer_12')
layer13 = conv(layer12, units=128, k=3, s=1, padding='SAME', activation=tf.nn.relu, name='layer_13')
max_pool5 = tf.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), name='maxpool_5')(layer13)

top_conv = tf.identity(max_pool5, 'top_conv') # 마지막 layer 을 top conv 에 넣습니다.

# Fully Connected Layer
- 설계한 fully connected layer 을 구현합니다.
- dropout 을 적용합니다 

In [0]:
def fc(flat_layer, units, activation, initializer, layer_name):
    n_input = int(flat_layer.get_shape()[-1])
    with tf.variable_scope(layer_name, reuse=tf.AUTO_REUSE):
        w = tf.get_variable(name='w', shape=[n_input, units], initializer=initializer())
        b = tf.get_variable(name='b', shape=[units], initializer=tf.initializers.constant(value=0, dtype=tf.float32))
        layer = tf.matmul(flat_layer, w) + b
    return layer

In [0]:
# flat layer 
flat_layer = tf.layers.flatten(top_conv)

# fully connected layer 1
fc_initalizer = tf.initializers.he_normal()
fc_layer_1 = fc(flat_layer, 512, tf.nn.relu, tf.initializers.he_normal, 'fc1')
fc_layer_1 = tf.layers.dropout(fc_layer_1, training=phase_train, rate=0.5)

fc_layer_2 = fc(fc_layer_1, 512, tf.nn.relu, tf.initializers.he_normal, 'fc2')
fc_layer_2 = tf.layers.dropout(fc_layer_2, training=phase_train, rate=0.5)

fc_layer_3 = fc(fc_layer_2, 10, None, tf.initializers.glorot_normal, 'fc3')
logits= tf.identity(fc_layer_3, 'logits')

W0616 11:37:21.352409 139677661910912 deprecation.py:323] From <ipython-input-7-0f18bd428d96>:1: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.
W0616 11:37:21.621757 139677661910912 deprecation.py:506] From <ipython-input-6-40e69d0e0264>:5: calling Constant.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0616 11:37:21.632240 139677661910912 deprecation.py:323] From <ipython-input-7-0f18bd428d96>:6: dropout (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dropout instead.
W0616 11:37:21.761134 139677661910912 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:128

#  Loss function 
- loss function 을 정의합니다. L2 regularization 을 사용합니다 

In [0]:
l2_reg = tf.reduce_sum([tf.nn.l2_loss(var) for var in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)])
l2_beta = 0.01

#loss 
# L2 regularization 
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=ys, logits=logits)
loss = tf.reduce_mean(loss, name='loss') + l2_reg*l2_beta

# Metric

In [0]:
# metric
logits_cls = tf.argmax(logits, axis=1)
logits_cls = tf.cast(logits_cls, dtype=tf.int32)
acc = tf.metrics.accuracy(labels=ys, predictions=logits_cls, name='accuracy')

# Add tensor to Tensorboard 

In [0]:
# add accuracy to tensorboard nodes 
tf.summary.scalar(name='accuracy', tensor=acc)

# add loss to tensorboard nodes 
tf.summary.scalar(name='loss', tensor=loss)

#merge all tensorboard nodes 
merged = tf.summary.merge_all()

# Optimizer 

In [0]:
# train_op : adamoptimizer 
train_op = tf.train.AdamOptimizer(lr).minimize(loss)

# Session open 

In [0]:
sess = tf.Session()

init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())# : globalal initializer 
sess.run(init)

# saver 
saver = tf.train.Saver()

# Tensorboard Filewriter 

In [0]:
# tensorboard 
train_writer=tf.summary.FileWriter(logdir='./log/train')
test_writer=tf.summary.FileWriter(logdir='./log/test')

# Training 

In [0]:
dataprovider = DataProvider(images=x_train, labels=y_train)
# save_root_folder = #fix me # : models saved folder 

# hparam 
batch_size = 100
min_loss = 1000000.0
learning_rate = 0.0005

for i in range(10000):
    batch_xs, batch_ys = dataprovider.next_batch(batch_size)
    # training 
    _ = sess.run(train_op, feed_dict={xs: batch_xs, ys: batch_ys, lr: learning_rate, phase_train: True})
    
    if i % 100 == 0 :
        # Validate validation dataset 
        fetches=[loss, acc, merged]
        val_loss, val_acc, val_merged = sess.run(fetches, feed_dict={xs: x_test, ys: y_test, phase_train: False})

        # Validate train dataset : extract randomly 10000 samples from train dataset 
        batch_xs, batch_ys = dataprovider.next_batch(10000)
        train_loss, train_acc, train_merged = sess.run(fetches, feed_dict={xs: batch_xs, ys: batch_ys, phase_train: False})
        
        print('step : {} train loss : {:.4f} acc : {:.4f} | Val loss : {:.4f} acc : {:.4f}'.\
        format(i, train_loss, train_acc, val_loss, val_acc))

#         # Save Model 
#         if #fix me # : when val_loss < min_loss 
#             min_loss = val_loss
#             save_path = #fix me #
#             saver.save(#fix me#)
#             print('model save!')
            
        # Add values to tensorboard 
        train_writer.add_summary(summary=train_merged, global_step=i)
        test_writer.add_summary(summary=val_merged, global_step=i)
        train_writer.flush()

InvalidArgumentError: ignored

<hr>
<div style = "background-image: url('https://algorithmai.io/static/media/logo.665798c4.png');background-repeat: no-repeat; background-position: right; background-size: 220px 40px; padding : 5px 10px 5px 5px;">
    Copyright(c) 2019 by Public AI. All rights reserved.<br>
    Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2019/06/17
</div>
<hr>